# Trabajando con Spark SQL y dataframes

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark= SparkSession.builder.appName("Trabajando con Spark SQL").getOrCreate()

Lo primero que vamos a leer va a ser un fichero json que representa la tabla periodica de los elementos y lo vamos a almacenar en un dataframe sobre el que vamos a ir realizando diferentes acciones como si se tratara de un RDD.


Nota: el formato json en spark SQL es un formato por línea , como si fuera un CSV, por lo tanto, hay que transformar el listado de objetos en un una fila por cada objeto.

In [3]:
df = spark.read.json("PeriodicTable.json")

In [4]:
df.collect()

[Row(appearance=u'colorless gas', atomic_mass=1.008, boil=20.271, category=u'diatomic nonmetal', color=None, density=0.08988, discovered_by=u'Henry Cavendish', melt=13.99, molar_heat=28.836, name=u'Hydrogen', named_by=u'Antoine Lavoisier', number=u'1', period=1, phase=u'Gas', source=u'https://en.wikipedia.org/wiki/Hydrogen', spectral_img=u'https://en.wikipedia.org/wiki/File:Hydrogen_Spectra.jpg', summary=u'Hydrogen is a chemical element with chemical symbol H and atomic number 1. With an atomic weight of 1.00794 u, hydrogen is the lightest element on the periodic table. Its monatomic form (H) is the most abundant chemical substance in the Universe, constituting roughly 75% of all baryonic mass.', symbol=u'H', xpos=1, ypos=1),
 Row(appearance=u'colorless gas, exhibiting a red-orange glow when placed in a high-voltage electric field', atomic_mass=4.0026022, boil=4.222, category=u'noble gas', color=None, density=0.1786, discovered_by=u'Pierre Janssen', melt=0.95, molar_heat=None, name=u'H

In [5]:
df.printSchema()

root
 |-- appearance: string (nullable = true)
 |-- atomic_mass: double (nullable = true)
 |-- boil: double (nullable = true)
 |-- category: string (nullable = true)
 |-- color: string (nullable = true)
 |-- density: double (nullable = true)
 |-- discovered_by: string (nullable = true)
 |-- melt: double (nullable = true)
 |-- molar_heat: double (nullable = true)
 |-- name: string (nullable = true)
 |-- named_by: string (nullable = true)
 |-- number: string (nullable = true)
 |-- period: long (nullable = true)
 |-- phase: string (nullable = true)
 |-- source: string (nullable = true)
 |-- spectral_img: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- xpos: long (nullable = true)
 |-- ypos: long (nullable = true)



In [6]:
df.select("category","name").show()

+--------------------+----------+
|            category|      name|
+--------------------+----------+
|   diatomic nonmetal|  Hydrogen|
|           noble gas|    Helium|
|        alkali metal|   Lithium|
|alkaline earth metal| Beryllium|
|           metalloid|     Boron|
| polyatomic nonmetal|    Carbon|
|   diatomic nonmetal|  Nitrogen|
|   diatomic nonmetal|    Oxygen|
|   diatomic nonmetal|  Fluorine|
|           noble gas|      Neon|
|        alkali metal|    Sodium|
|alkaline earth metal| Magnesium|
|post-transition m...| Aluminium|
|           metalloid|   Silicon|
| polyatomic nonmetal|Phosphorus|
| polyatomic nonmetal|    Sulfur|
|   diatomic nonmetal|  Chlorine|
|           noble gas|     Argon|
|        alkali metal| Potassium|
|alkaline earth metal|   Calcium|
+--------------------+----------+
only showing top 20 rows



### Seleccionamos los elementos químicos que tengan la masa atómica menor que 200 y mostramos los 10 primeros.

In [7]:
df.select(df['name'],df['atomic_mass']).filter(df['atomic_mass']<200).show(10)

+---------+-------------+
|     name|  atomic_mass|
+---------+-------------+
| Hydrogen|        1.008|
|   Helium|    4.0026022|
|  Lithium|         6.94|
|Beryllium|   9.01218315|
|    Boron|        10.81|
|   Carbon|       12.011|
| Nitrogen|       14.007|
|   Oxygen|       15.999|
| Fluorine|18.9984031636|
|     Neon|     20.17976|
+---------+-------------+
only showing top 10 rows



In [8]:
#Agrupar por categoria
df.groupBy('category').count().show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|unknown, probably...|    4|
|unknown,probably ...|    1|
|        alkali metal|    6|
|          lanthanide|   15|
|   diatomic nonmetal|    7|
|    transition metal|   35|
|unknown, probably...|    2|
| polyatomic nonmetal|    4|
|            actinide|   15|
|           metalloid|    7|
|alkaline earth metal|    6|
|           noble gas|    6|
|unknown, predicte...|    1|
|post-transition m...|    9|
+--------------------+-----+



### Ahora vamos a ver como a partir de un dataframe podemos generar una tabla temporal sobre la que ejecutaremos sentencias en SQL.

In [9]:
df.createGlobalTempView("TablaPeriodica")

In [10]:
spark.sql("select category,name from global_temp.TablaPeriodica").show(10)

+--------------------+---------+
|            category|     name|
+--------------------+---------+
|   diatomic nonmetal| Hydrogen|
|           noble gas|   Helium|
|        alkali metal|  Lithium|
|alkaline earth metal|Beryllium|
|           metalloid|    Boron|
| polyatomic nonmetal|   Carbon|
|   diatomic nonmetal| Nitrogen|
|   diatomic nonmetal|   Oxygen|
|   diatomic nonmetal| Fluorine|
|           noble gas|     Neon|
+--------------------+---------+
only showing top 10 rows



### La creación de dataset se basa en la definición de una clase y permite añadir objetos de esa clase. El resultado es una estructura en formato de tabla como el dataframe mostrado en nuestro caso.

## Generar el esquema a partir de un fichero de texto

En Spark SQL, existen dos formas de generar el esquema de un dataframe. Una es mediante reflexión y la otra es explicitamente con programación. A continuación vamos a ver ambos casos sobre un documento txt que contiene el elemento químico y su masa atómica.

In [19]:
from pyspark.sql import Row
sc = spark.sparkContext
lines=sc.textFile("tablaPeriodica.txt")
parts= lines.map(lambda p: p.split(","))

elementos= parts.map(lambda e: Row(nombre=e[0],masa_atomica=float(e[1])))
elementos.count()

10

### Generar el esquema

In [20]:
esquema=spark.createDataFrame(elementos)
esquema.createOrReplaceTempView("elementos")
esquema.printSchema()

root
 |-- masa_atomica: double (nullable = true)
 |-- nombre: string (nullable = true)



In [21]:
consultaElementos=spark.sql("select nombre,masa_atomica from elementos where masa_atomica>0 and masa_atomica<21")

In [22]:
data=consultaElementos.rdd.map(lambda elem: "Nombre: "+elem.nombre).collect()
for name in data:
    print(name)

Nombre: Hydrogen
Nombre: Helium
Nombre: Lithium
Nombre: Beryllium
Nombre: Boron
Nombre: Carbon
Nombre: Nitrogen
Nombre: Oxygen
Nombre: Fluorine
Nombre: Neon


Ahora vamos a ver como se haría programáticamente.

In [29]:
from pyspark.sql.types import *

sc=spark.sparkContext

lines=sc.textFile("tablaPeriodica.txt")
parts=lines.map(lambda line: line.split(","))
elementos= parts.map(lambda p: (p[0],p[1]))

campos= [StructField("nombre",StringType(),False),StructField("masaAtomica",StringType(),False)]

#crear esquema a partir de la estructura de campos
esquema =StructType(campos)

esquemaElementos= spark.createDataFrame(elementos,esquema)

esquemaElementos.createOrReplaceTempView("elementos")

esquemaElementos.printSchema()

root
 |-- nombre: string (nullable = false)
 |-- masaAtomica: string (nullable = false)



In [30]:
print(esquemaElementos.collect())

[Row(nombre=u'Hydrogen', masaAtomica=u'1.008'), Row(nombre=u'Helium', masaAtomica=u'4.0026022'), Row(nombre=u'Lithium', masaAtomica=u'6.94'), Row(nombre=u'Beryllium', masaAtomica=u'9.01218315'), Row(nombre=u'Boron', masaAtomica=u'10.81'), Row(nombre=u'Carbon', masaAtomica=u'12.011'), Row(nombre=u'Nitrogen', masaAtomica=u'14.007'), Row(nombre=u'Oxygen', masaAtomica=u'15.999'), Row(nombre=u'Fluorine', masaAtomica=u'18.9984031636'), Row(nombre=u'Neon', masaAtomica=u'20.17976')]


In [32]:
spark.sql("select nombre,masaAtomica from elementos").show()

+---------+-------------+
|   nombre|  masaAtomica|
+---------+-------------+
| Hydrogen|        1.008|
|   Helium|    4.0026022|
|  Lithium|         6.94|
|Beryllium|   9.01218315|
|    Boron|        10.81|
|   Carbon|       12.011|
| Nitrogen|       14.007|
|   Oxygen|       15.999|
| Fluorine|18.9984031636|
|     Neon|     20.17976|
+---------+-------------+

